In [1]:
import yaml

import awkward as ak
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import mplhep as hep

import hist
from hist import Hist

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from topcoffea.modules.histEFT import HistEFT
NanoAODSchema.warn_missing_crossrefs = False

from coffea.analysis_tools import PackedSelection
from topcoffea.modules import utils
import topcoffea.modules.eft_helper as efth

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import dctr.modules.plotting_tools as plt_tools
import dctr.modules.DNN_tools as DNN_tools

In [2]:
fname_powheg = "/cms/cephfs/data/store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/74C36AED-4CB9-1A4D-A9E6-90278C68131C.root"
# Load in events from root file
events_powheg = NanoEventsFactory.from_root(
    fname_powheg,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "TTto2L2Nu"},
).events()

fname_smeft = "/cms/cephfs/data/store/user/hnelson2/noEFT/nanoGen/TT01j2l_SM/NanoGen_TT01j2l_SM/nanoGen_10016.root"
# Load in events from root file
events_smeft = NanoEventsFactory.from_root(
    fname_smeft,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "TTto2L2Nu"},
).events()

In [3]:
# means and stdv to standardize pd df for input into trained model

means = {'avg_top_pt': 34.263557,
        'mtt': 522.141900,
        'top1pt': 126.859184,
        'top1eta': -0.257265,
        'top1phi': -0.000021,
        'top1mass': 172.253560,
        'top2pt': 124.636566,
        'top2eta': 0.257370,
        'top2phi': -0.000686,
        'top2mass': 172.265670,
}

stdvs = {'avg_top_pt': 38.252880,
        'mtt': 175.306980,
        'top1pt': 84.604750,
        'top1eta': 1.823326,
        'top1phi': 1.813635,
        'top1mass': 5.346320,
        'top2pt': 82.644310,
        'top2eta': 1.829129,
        'top2phi': 1.813916,
        'top2mass': 5.329451,}

In [ ]:
axes = {
    "outputs": {
        "regular": (100, 0, 1),
        "label": "outputs",
    },
    "lep1pt": {
        "regular": (40, 0, 400),
        "label": "lep1 pt",
    },
    "lep2pt": {
        "regular": (40, 0, 400),
        "label": "lep2 pt",
    },
    "lpluspt": {
        "regular": (40, 0, 400),
        "label": "lplus pt",
    },
    "lminuspt": {
        "regular": (40, 0, 400),
        "label": "lminus pt",
    },
    "top1pt": {
        "regular": (35, 0, 700),
        "label": "top1 pt",
    },
    "top2pt": {
        "regular": (35, 0, 700),
        "label": "top2 pt",
    },
    "toppt": {
        "regular": (35, 0, 700),
        "label": "top pt",
    },
    "antitoppt": {
        "regular": (35, 0, 700),
        "label": "antitop pt",
    },
    
    "lep1eta": {
        "regular": (50, -5, 5),
        "label": "lep1 eta",
    },
    "lep2eta": {
        "regular": (50, -5, 5),
        "label": "lep2 eta",
    },
    "lpluseta": {
        "regular": (50, -5, 5),
        "label": "lplus eta",
    },
    "lminuseta": {
        "regular": (50, -5, 5),
        "label": "lminus eta",
    },

    "top1eta": {
        "regular": (50, -5, 5),
        "label": "top1 eta",
    },
    "lep2eta": {
        "regular": (50, -5, 5),
        "label": "lep2 eta",
    },
    "lpluseta": {
        "regular": (50, -5, 5),
        "label": "lplus eta",
    },
    "lminuseta": {
        "regular": (50, -5, 5),
        "label": "lminus eta",
    },
    
    "lep1pt": {
        "regular": (40, -4, 4),
        "label": "lep1 pt",
    },
    "lep2pt": {
        "regular": (40, -4, 4),
        "label": "lep2 pt",
    },
    "lpluspt": {
        "regular": (40, -4, 4),
        "label": "lplus pt",
    },
    "lminuspt": {
        "regular": (40, -4, 4),
        "label": "lminus pt",
    },
    "lep1mass": {
        "regular": (20, 0, 20),
        "label": "lep1 mass", 
    },
    "lep2mass": {
        "regular": (20, 0, 20),
        "label": "lep2 mass", 
    },
    "lplusmass": {
        "regular": (20, 0, 20),
        "label": "lplus mass", 
    },
    "lminusmass": {
        "regular": (20, 0, 20),
        "label": "lminus mass", 
    },  
}

## Create objects for plotting/selections

In [6]:
genpart = events_powheg.GenPart
is_final_mask = genpart.hasFlags(["fromHardProcess","isLastCopy"])

In [7]:
gen_top = ak.pad_none(genpart[is_final_mask & (abs(genpart.pdgId) == 6)],2)
gen_top = gen_top[ak.argsort(gen_top.pt, axis=1, ascending=False)]

top = genpart[is_final_mask & (genpart.pdgId == 6)]
antitop = genpart[is_final_mask & (genpart.pdgId == -6)]

In [8]:
ele  = genpart[is_final_mask & (abs(genpart.pdgId) == 11)]
mu   = genpart[is_final_mask & (abs(genpart.pdgId) == 13)]
tau = genpart[is_final_mask & (abs(genpart.pdgId) == 15)]

leps = ak.concatenate([ele, mu, tau],axis=1)
leps = leps[ak.argsort(leps.pt, axis=-1, ascending=False)]

lplus = leps[leps.pdgId < 0] #negative pdgId corresponds with antielectorn, antimuon, antitau
lminus = lminus = leps[leps.pdgId > 0] #positive pdgId corresponds with electron, muon, tau 

In [10]:
jets = events_powheg.GenJet
jets = jets[ak.argsort(jets.pt, axis=-1, ascending=False)]
njets = ak.num(jets)

## Fill df with inputs to run through DNN

In [11]:
variables_to_fill_df = {
    "avg_top_pt": np.divide(gen_top.sum().pt, 2.0),
    "mtt"       : (gen_top[:,0] + gen_top[:,1]).mass,
    "top1pt"    : gen_top.pt[:,0],
    "top1eta"   : gen_top.eta[:,0],
    "top1phi"   : gen_top.phi[:,0],
    "top1mass"  : gen_top.mass[:,0],
    "top2pt"    : gen_top.pt[:,1],
    "top2eta"   : gen_top.eta[:,1],
    "top2phi"   : gen_top.phi[:,1],
    "top2mass"  : gen_top.mass[:,1],
}

# NN_inputs = pd.DataFrame.from_dict(variables_to_fill_df)
norm_NN_inputs = DNN_tools.standardize_df(pd.DataFrame.from_dict(variables_to_fill_df), means, stdvs)

## Load in trained network, run powheg data through

In [13]:
config_path = "/users/hnelson2/dctr/condor_submissions/20250721_1722/config.yaml"
with open(config_path, 'r') as f:
        config_dict = yaml.safe_load(f)

In [14]:
model_architecture = config_dict['model']
input_dim = norm_NN_inputs.shape[1]
model = DNN_tools.NeuralNetwork(input_dim, model_architecture)

In [15]:
model_path = "/users/hnelson2/dctr/condor_submissions/20250721_1722/training_outputs/final_model.pt"
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [16]:
model.eval()
predictions = DNN_tools.get_predictions(model, torch.from_numpy(norm_NN_inputs.to_numpy()))

In [17]:
output_mask = predictions<0.1

In [ ]:


histos_powheg = {
    "outputs" : Hist(hist.axis.Regular(bins=100, start=0, stop=1, name="outputs")),
    "lep1pt": Hist(hist.axis.Regular(bins=40, start=0, stop=400, name="lep1pt")),
    "lep1eta": Hist(hist.axis.Regular(bins=50, start=-5, stop=5, name="lep1eta")),
    "lep1phi": Hist(hist.axis.Regular(bins=40, start=-4, stop=4, name="lep1phi")),
    "njets": Hist(hist.axis.Regular(bins=10, start=0, stop=10, name="njets")),
    "j0pt": Hist(hist.axis.Regular(bins=40, start=0, stop=400, name="j0pt")),
    }

histos_pohweg_cuts = {
    "outputs" : Hist(hist.axis.Regular(bins=100, start=0, stop=1, name="outputs")),
    "lep1pt": Hist(hist.axis.Regular(bins=40, start=0, stop=400, name="lep1pt")),
    "lep1eta": Hist(hist.axis.Regular(bins=50, start=-5, stop=5, name="lep1eta")),
    "lep1phi": Hist(hist.axis.Regular(bins=40, start=-4, stop=4, name="lep1phi")),
    "njets": Hist(hist.axis.Regular(bins=10, start=0, stop=10, name="njets")),
    "j0pt": Hist(hist.axis.Regular(bins=40, start=0, stop=400, name="j0pt")),
    }

histos_smeft = {
    "outputs" : Hist(hist.axis.Regular(bins=100, start=0, stop=1, name="outputs")),
    "lep1pt": Hist(hist.axis.Regular(bins=40, start=0, stop=400, name="lep1pt")),
    "lep1eta": Hist(hist.axis.Regular(bins=50, start=-5, stop=5, name="lep1eta")),
    "lep1phi": Hist(hist.axis.Regular(bins=40, start=-4, stop=4, name="lep1phi")),
    "njets": Hist(hist.axis.Regular(bins=10, start=0, stop=10, name="njets")),
    "j0pt": Hist(hist.axis.Regular(bins=40, start=0, stop=400, name="j0pt")),
    }

In [ ]:
variables_to_fill_cuts = {
    "outputs": predictions, 
    "lep1pt": leps.pt[:,0][output_mask],
    "lep1eta": leps.eta[:,0][output_mask],
    "lep1phi": leps.phi[:,0][output_mask],
    "njets": njets[output_mask],
    }

for var_name, var_val in variables_to_fill_cuts.items():
    histos_cuts[var_name].fill(**{var_name: var_val})

In [ ]:
variables_to_fill = {
    "outputs": predictions, 
    "lep1pt": leps.pt[:,0],
    "lep1eta": leps.eta[:,0],
    "lep1phi": leps.phi[:,0],
    "njets": ak.num(jets),
    }

for var_name, var_val in variables_to_fill.items():
    histos[var_name].fill(**{var_name: var_val})

In [ ]:
for name in histos: 
    fig, ax = plt.subplots()
    hep.histplot(histos[name], density=True, label='no cuts')
    hep.histplot(histos_cuts[name], density=True, label='cuts')
    ax.legend(loc='upper right')

In [ ]:
leps.pt[:,0][output_mask]

In [ ]:
leps.pt[:,0]